# Machine Learning for Trees

In [1]:
# special IPython command to prepare the notebook for matplotlib
%matplotlib inline 

import requests 
from StringIO import StringIO
import numpy as np
import pandas as pd # pandas
import matplotlib.pyplot as plt # module for plotting 
import datetime as dt # module for manipulating dates and times
import numpy.linalg as lin # module for performing linear algebra operations
import collections
from sklearn.model_selection import train_test_split

In [2]:
# special matplotlib argument for improved plots
from matplotlib import rcParams

#colorbrewer2 Dark2 qualitative color table
dark2_colors = [(0.10588235294117647, 0.6196078431372549, 0.4666666666666667),
                (0.8509803921568627, 0.37254901960784315, 0.00784313725490196),
                (0.4588235294117647, 0.4392156862745098, 0.7019607843137254),
                (0.9058823529411765, 0.1607843137254902, 0.5411764705882353),
                (0.4, 0.6509803921568628, 0.11764705882352941),
                (0.9019607843137255, 0.6705882352941176, 0.00784313725490196),
                (0.6509803921568628, 0.4627450980392157, 0.11372549019607843)]

rcParams['figure.figsize'] = (10, 6)
rcParams['figure.dpi'] = 150
rcParams['axes.color_cycle'] = dark2_colors
rcParams['lines.linewidth'] = 2
rcParams['axes.facecolor'] = 'white'
rcParams['font.size'] = 14
rcParams['patch.edgecolor'] = 'white'
rcParams['patch.facecolor'] = dark2_colors[0]
rcParams['font.family'] = 'StixGeneral'

/Users/rachellim/anaconda2/lib/python2.7/site-packages/matplotlib/__init__.py:913: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


## 1. Read in tree data

In [3]:
tree_url = "https://data.cityofnewyork.us/api/views/uvpi-gqnh/rows.csv?accessType=DOWNLOAD"
tree = pd.read_csv(tree_url)
tree.head()

,tree_id,block_id,created_at,tree_dbh,stump_diam,curb_loc,status,health,spc_latin,spc_common,...,boro_ct,state,latitude,longitude,x_sp,y_sp,council district,census tract,bin,bbl
0,180683,348711,08/27/2015,3,0,OnCurb,Alive,Fair,Acer rubrum,red maple,...,4073900,New York,40.723092,-73.844215,1027431.148,202756.7687,29.0,739.0,4052307.0,4.022210e+09
1,200540,315986,09/03/2015,21,0,OnCurb,Alive,Fair,Quercus palustris,pin oak,...,4097300,New York,40.794111,-73.818679,1034455.701,228644.8374,19.0,973.0,4101931.0,4.044750e+09
2,204026,218365,09/05/2015,3,0,OnCurb,Alive,Good,Gleditsia triacanthos var. inermis,honeylocust,...,3044900,New York,40.717581,-73.936608,1001822.831,200716.8913,34.0,449.0,3338310.0,3.028870e+09
3,204337,217969,09/05/2015,10,0,OnCurb,Alive,Good,Gleditsia triacanthos var. inermis,honeylocust,...,3044900,New York,40.713537,-73.934456,1002420.358,199244.2531,34.0,449.0,3338342.0,3.029250e+09
4,189565,223043,08/30/2015,21,0,OnCurb,Alive,Good,Tilia americana,American linden,...,3016500,New York,40.666778,-73.975979,990913.775,182202.4260,39.0,165.0,3025654.0,3.010850e+09


## 2. Most common tree type

In [4]:
tree10 = tree.groupby('spc_common')['tree_id'].nunique().sort_values(ascending=False).head(10)


In [5]:
tree10 = ['London planetree', 'honeylocust', 'Callery pear', 'pin oak', 'Norway maple', 'littleleaf linden', 'cherry', 'Japanese zelkova', 'ginkgo', 'Sophora']

In [6]:
tree10 = tree.loc[tree['spc_common'].isin(tree10)]

In [7]:
tree10.shape

(426224, 45)

## 3. Check general health of trees

In [8]:
tree.groupby('health')['tree_id'].nunique()

health
Fair     96504
Good    528850
Poor     26818
Name: tree_id, dtype: int64

## 4. Identify underlying pattern in tree health

In [9]:
tree_health = tree.groupby(['spc_common', 'health'])['tree_id'].count()
tree_health_pct = tree_health.groupby(level=[0]).apply(lambda x: x / x.sum())


In [10]:
tree_health_pct = tree_health_pct.to_frame().reset_index()

In [11]:
tree_health_pct.head()

,spc_common,health,tree_id
0,'Schubert' chokecherry,Fair,0.146686
1,'Schubert' chokecherry,Good,0.803805
2,'Schubert' chokecherry,Poor,0.049509
3,American beech,Fair,0.139194
4,American beech,Good,0.783883


Tree types with the highest percentage of trees classified as poor, fair or good

In [12]:
tree_health_pct[tree_health_pct['tree_id'] == tree_health_pct.groupby(['health'])['tree_id'].transform(max)]


,spc_common,health,tree_id
132,Virginia pine,Poor,0.200000
157,black pine,Fair,0.432432
292,pitch pine,Good,1.000000


In [13]:
tree10.columns

Index([u'tree_id', u'block_id', u'created_at', u'tree_dbh', u'stump_diam',
       u'curb_loc', u'status', u'health', u'spc_latin', u'spc_common',
       u'steward', u'guards', u'sidewalk', u'user_type', u'problems',
       u'root_stone', u'root_grate', u'root_other', u'trunk_wire',
       u'trnk_light', u'trnk_other', u'brch_light', u'brch_shoe',
       u'brch_other', u'address', u'postcode', u'zip_city', u'community board',
       u'borocode', u'borough', u'cncldist', u'st_assem', u'st_senate', u'nta',
       u'nta_name', u'boro_ct', u'state', u'latitude', u'longitude', u'x_sp',
       u'y_sp', u'council district', u'census tract', u'bin', u'bbl'],
      dtype='object')

In [14]:
#remove unneccessary columns
tree10_select = tree10[['tree_id','tree_dbh', 'stump_diam', 'curb_loc', 'status', 'health', 'spc_common', 'sidewalk', 'user_type', 'root_stone', 'root_grate', 'root_other', 'trunk_wire', 'trnk_light', 'trnk_other', 'brch_light', 'brch_shoe', 'brch_other']]

In [15]:
tree10_select.shape

(426224, 18)

In [16]:
data=tree10_select.dropna()
data.shape

(426222, 18)

In [17]:
# remove records with any missing values
data=tree10_select.dropna()


y=data.loc[:,"health"]

X=data.loc[:, (data.columns != "health") & (data.columns != "tree_id")]
X=pd.get_dummies(X)

# Split data into 70% train, 30% test
X_train,X_test,y_train,y_test=train_test_split(X, y, test_size=0.3, random_state=999)
print X_train.head()

        tree_dbh  stump_diam  curb_loc_OffsetFromCurb  curb_loc_OnCurb  \
663269        26           0                        0                1   
368187        32           0                        0                1   
573028        17           0                        0                1   
252652         8           0                        0                1   
185049        15           0                        0                1   

        status_Alive  spc_common_Callery pear  spc_common_Japanese zelkova  \
663269             1                        0                            0   
368187             1                        0                            0   
573028             1                        0                            0   
252652             1                        0                            0   
185049             1                        0                            0   

        spc_common_London planetree  spc_common_Norway maple  \
663269                

In [18]:
from sklearn.tree import DecisionTreeClassifier

# learn model
dt=DecisionTreeClassifier()
dt.fit(X_train,y_train)

# in sample accuracy
print 'In sample accuracy:',dt.score(X_train,y_train)

# out of sample accuracy
print 'Out of sample accuracy:',dt.score(X_test,y_test)

In sample accuracy: 0.839566958824
Out of sample accuracy: 0.808965565783


In [19]:
OS=[]
for i in range(10):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = i)    
    dt = DecisionTreeClassifier()
    dt.fit(X_train,y_train)
    OS.append(dt.score(X_test,y_test))
print np.mean(OS)

0.806613121447


In [20]:
from sklearn.model_selection import GridSearchCV
param_grid ={'max_depth':range(1,11)}
dt = DecisionTreeClassifier()
gr=GridSearchCV(dt,param_grid=param_grid)
rs=gr.fit(X_train,y_train)
print 'Best parameter value:',rs.best_params_
y_predict = rs.predict(X_test)
print 'Accuracy = ',((y_predict == y_test).value_counts(normalize=True)[True])

Best parameter value: {'max_depth': 6}
Accuracy =  0.821040612511


In [21]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_predict)

In [22]:
cm

array([[   353,  17726,    143],
       [   320, 104468,     72],
       [   198,   4424,    163]])

In [23]:
dt = DecisionTreeClassifier(max_depth=6)
dt.fit(X_train, y_train)
Feature_importance=pd.DataFrame([list(X_train.columns),list(dt.feature_importances_)]).T
Feature_importance.columns=["variables","importance"]

# list the top 5 most important features in order
Feature_importance.sort_values(by="importance",ascending=False).iloc[:5,:]

,variables,importance
8,spc_common_Norway maple,0.324333
36,brch_other_No,0.296939
0,tree_dbh,0.118564
30,trnk_other_No,0.107663
17,user_type_NYC Parks Staff,0.0645483


## SVM Classifier

In [ ]:
# training a linear SVM classifier
from sklearn.svm import SVC
svm_model_linear = SVC(kernel = 'linear', C = 1).fit(X_train, y_train)
svm_predictions = svm_model_linear.predict(X_test)
 
# model accuracy for X_test  
accuracy = svm_model_linear.score(X_test, y_test)
print accuracy

# creating a confusion matrix
cm = confusion_matrix(y_test, svm_predictions)
print cm

## KNN Classifier

In [27]:
# training a KNN classifier
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = 7).fit(X_train, y_train)
 
# accuracy on X_test
accuracy = knn.score(X_test, y_test)
print accuracy
 
# creating a confusion matrix
knn_predictions = knn.predict(X_test) 
cm = confusion_matrix(y_test, knn_predictions)
print cm

0.802763809271
[[  1630  16372    220]
 [  3691 100848    321]
 [   654   3962    169]]


## NB Classifier

In [26]:
# training a Naive Bayes classifier
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB().fit(X_train, y_train)
gnb_predictions = gnb.predict(X_test)
 
# accuracy on X_test
accuracy = gnb.score(X_test, y_test)
print accuracy
 
# creating a confusion matrix
cm = confusion_matrix(y_test, gnb_predictions)
print cm

0.749192520353
[[ 1889 13872  2461]
 [ 6997 92935  4928]
 [  381  3431   973]]
